In [8]:
!pip install py-enigma


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from enigma.machine import EnigmaMachine

In [10]:
def find_all_cycles(graph, k = 3):
    def dfs(node, start_node, visited, path, all_cycles):
        path.append(node)
        visited[node] = True

        for neighbor in graph[node]:
            if neighbor == start_node and len(path) > 2:
                # Found circle: Copy current path and add it to the list
                all_cycles.append(path[:])
            elif neighbor not in visited or not visited[neighbor]:
                # Visit the neighbor who has not been visited yet
                dfs(neighbor, start_node, visited, path, all_cycles)

        # Backtrack: Remove current node from path and mark it as visited
        path.pop()
        visited[node] = False

    all_cycles = []
    visited = {node: False for node in graph}

    for start_node in graph:
        dfs(start_node, start_node, visited, [], all_cycles)
    

    # Remove duplicates
    unique_cycles = []
    seen = list()
    for cycle in all_cycles:
        cycle_set = set(cycle)
        if cycle_set not in seen:
            unique_cycles.append(cycle)
            seen.append(cycle_set)

    unique_cycles.sort(key=len, reverse=True)

    return unique_cycles[0: min(k, len(unique_cycles))]

# example graph
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'C', 'D'],
    'C': ['A', 'B', 'E'],
    'D': ['B', 'E'],
    'E': ['C', 'D']
}

graph = {'A': ['T', 'L', 'E', 'I'], 'B': [], 'C': ['E', 'T', 'E'], 'D': ['V', 'G'], 'E': ['C', 'X', 'M', 'A', 'X', 'T', 'C'], 'F': ['M'], 'G': ['S', 'H', 'D'], 'H': ['G'], 'I': ['Z', 'V', 'A', 'Q'], 'J': ['N', 'R'], 'K': [], 'L': ['A', 'R'], 'M': ['R', 'E', 'F'], 'N': ['J', 'V', 'R', 'R'], 'O': [], 'P': [], 'Q': ['I'], 'R': ['M', 'N', 'L', 'N', 'J'], 'S': ['G', 'U', 'Y'], 'T': ['Z', 'V', 'A', 'C', 'E'], 'U': ['S', 'X'], 'V': ['T', 'N', 'I', 'D'], 'W': [], 'X': ['E', 'E', 'U'], 'Y': ['S'], 'Z': ['T', 'I']}


# Suche nach allen Kreisen im Graphen
all_cycles = find_all_cycles(graph, 3)
print("Alle Kreise:")
for cycle in all_cycles:
    print(" -> ".join(cycle))


Alle Kreise:
A -> L -> R -> J -> N -> V -> D -> G -> S -> U -> X -> E -> C -> T -> Z -> I
A -> L -> R -> N -> V -> D -> G -> S -> U -> X -> E -> C -> T -> Z -> I
A -> L -> R -> J -> N -> V -> D -> G -> S -> U -> X -> E -> T -> Z -> I


In [11]:
def format_plugboard(input_string: str) -> str:
    """
    Formats a plugboard configuration from a given string.
    
    - Removes connections with duplicate letters (e.g., 'VV').
    - Sorts the letters in each connection alphabetically (e.g., 'WV' -> 'VW').
    - Removes duplicate connections (e.g., 'IU' appearing twice).
    - Returns the valid connections as a string separated by spaces.
    
    :param input_string: A string with plug pairs, e.g., 'WA QQ FT UH ZG VV LM WW BN SE DR AA'.
    :return: A formatted string of valid connections, e.g., 'AW FT HU GZ LM BN DR'.
    """
    # Split the string into pairs
    pairs = input_string.split()
    
    # Set to store valid connections (Set automatically avoids duplicates)
    formatted_pairs = set()
    
    for pair in pairs:
        # Skip pairs with duplicate letters
        if pair[0] == pair[1]:
            continue
        
        # Sort the letters alphabetically and add the pair to the set
        formatted_pairs.add("".join(sorted(pair)))
    
    # Join the pairs with spaces and return the string
    return " ".join(sorted(formatted_pairs))  # Sort for consistent output


# Example call
input_string = "AS GH IU OP TZ IU ER DF"
formatted = format_plugboard(input_string)
print(formatted)  # Output: "AS DF ER GH IU OP TZ"

AS DF ER GH IU OP TZ


In [12]:
def check_and_merge_plugboards(plugboard1, plugboard2):
    """
    Checks if two plugboard configurations are compatible and creates a merged plugboard.

    :param plugboard1: First plugboard as a string (e.g., "AW FT HU GZ")
    :param plugboard2: Second plugboard as a string (e.g., "AW FT LM BN")
    :return: (True, merged_plugboard) if compatible, otherwise (False, "")
    """
    def parse_plugboard(plugboard):
        mapping = {}
        for pair in plugboard.split():
            if len(pair) == 2:
                a, b = sorted(pair)  # Ensure the smaller letter comes first
                mapping[a] = b
                mapping[b] = a
        return mapping

    # Parse the plugboards into mappings
    mapping1 = parse_plugboard(plugboard1)
    mapping2 = parse_plugboard(plugboard2)

    merged_mapping = {}

    # Check for compatibility
    for char in set(mapping1.keys()).union(mapping2.keys()):
        if char in mapping1 and char in mapping2:
            # If the character is in both mappings, the values must match
            if mapping1[char] != mapping2[char]:
                return False, ""  # Conflict found
            else:
                merged_mapping[char] = mapping1[char]
        elif char in mapping1:
            merged_mapping[char] = mapping1[char]
        elif char in mapping2:
            merged_mapping[char] = mapping2[char]

    # Check if the number of connections exceeds 10
    unique_pairs = {frozenset([k, v]) for k, v in merged_mapping.items()}
    if len(unique_pairs) > 10:
        return False, ""  # Too many connections

    # Create the merged plugboard as a string
    merged_plugboard = " ".join(f"{min(k, v)}{max(k, v)}" for k, v in merged_mapping.items() if k < v)

    return True, format_plugboard(merged_plugboard)


# Example calls
plugboard1 = "AW FH HU GZ"
plugboard2 = "AW FT LM BN"

is_compatible, merged = check_and_merge_plugboards(plugboard1, plugboard2)
if is_compatible:
    print("Compatible plugboards:", merged)
else:
    print("Plugboards are not compatible.")

print(check_and_merge_plugboards('AB CD EF GH IJ KL MN OP QR ST', 'AB'))

Plugboards are not compatible.
(True, 'AB CD EF GH IJ KL MN OP QR ST')


In [13]:
def find_enigma_display_positions_to_index(rotor_string: str, plain_text: str, start_display:str):
    """This method determines the enigma display setting for every edge in the graph

    Args:
        rotor_string (str): 
        plain_text (str):
        start_display (str):

    Returns:
        array: contains the display settings for every edge
    """
    result_array = [""] * len(plain_text)
    enigma = EnigmaMachine.from_key_sheet(
        rotors=rotor_string,
        reflector='B',
        ring_settings="A A A",
        plugboard_settings='')
    enigma.set_display(start_display)
    for i in range(0, len(plain_text)): #iterate through every letter in the plain text := every edge in the graph
        result_array[i] = enigma.get_display()
        enigma.key_press('A')
    return result_array

find_enigma_display_positions_to_index('I II III', 'ABCDEFGHIJ', 'AEU')


['AEU', 'BFV', 'BGW', 'BGX', 'BGY', 'BGZ', 'BGA', 'BGB', 'BGC', 'BGD']

In [14]:
def check_plugboard_configurations(config_param_list: list):
    """This method checks if the plugboard configurations from the turing-bombe run with the same settings but on different cycles are compatible
    Removes incompatible plugboard-configurations

    Args:
        config_param_list (list): list, containing a list for every cycle. This list (in the list) contains all the reconstructed plugboards for each cycle

    Returns:
        list: Reduces the list of lists to a list with only one list in it - the plugboard settings that are compatible with all the cycles
    """
    config_list = config_param_list
    if(len(config_list) < 1):
        return False, list()
    while(len(config_list) > 1):
        result_list = list()
        for i in range(0, len(config_list[0])): #iterate through all plugboards from first cycle
            for j in range(0, len(config_list[1])): #iterate through all plugboards from second cycle
                valid, merged_plugboard = check_and_merge_plugboards(config_list[0][i], config_list[1][j]) #check if they are compatible and merge them
                if valid:
                    result_list.append(format_plugboard(merged_plugboard))
                    #print('MERGED: ' + merged_plugboard)
        config_list[0] = result_list #set the result as the first list item
        config_list.pop(1) #remove the second list - whose information is now contained in the first list
        
    return len(config_list[0]) > 0, config_list


print(check_plugboard_configurations(list([list(['AB', 'CD']), list(['EF'])])))
print(check_plugboard_configurations(list([list(['AB', 'AF']), list(['EF', 'AG'])])))
print(check_plugboard_configurations(list([list(['AB', 'AF']), list(['AF', 'AG'])])))
print(check_plugboard_configurations(list([list(['AB', 'AF']), list(['AG'])])))
print(check_plugboard_configurations(list([list(['AB', 'AF']), list()])))
print(check_plugboard_configurations(list([list()])))



(True, [['AB EF', 'CD EF']])
(True, [['AB EF']])
(True, [['AF']])
(False, [[]])
(False, [[]])
(False, [[]])


In [15]:
from itertools import combinations
from itertools import permutations

alphabet = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

def filter_permutations(permutations_list):
    """This method used by the method brute forcing the remaining plugboard filters out all duplicate versions of additional plugboard connections

    Args:
        permutations_list (list): list of permutations

    Returns:
        list: reduced list of permutations
    """
    return [perm for perm in permutations_list if format_plugboard(concatenate_with_spaces(perm)) == concatenate_with_spaces(perm)]

def concatenate_with_spaces(letters):
    # concatenate letters from list and add a space after every second letter
    result = "".join(
        letter + (" " if (i + 1) % 2 == 0 and i + 1 != len(letters) else "")
        for i, letter in enumerate(letters)
    )
    return result


def brute_force_remaining_plugboard(config: dict, cypher_text:str, plain_text:str):
    """This method tries out all remaining possibilities for the plugboard connections, based on what we found out from the loop structures

    Args:
        config (dict): dictionary containing all the found out parameters for this configuration
        cypher_text (str): the cypher_text
        plain_text (str): the plain text
    """
    plugboard_without_spaces = config['plugboard'].replace(" ", "")
    if(len(plugboard_without_spaces) < 12): 
        return #prevents kernel from crashing because of too many options
    unplugged_letters = [item for item in alphabet if not item in list(plugboard_without_spaces)]
    all_combinations = list(combinations(unplugged_letters, 20 - len(plugboard_without_spaces)))
    for combination in all_combinations:
        relevant_permutations = filter_permutations(list(permutations(combination)))
        for permutation in relevant_permutations: #try out all combinations and permutations of the remaining letters as plugboard connections
            try_out_plugboard = check_and_merge_plugboards(concatenate_with_spaces(permutation), config['plugboard'])[1]
            decypher_enigma = EnigmaMachine.from_key_sheet( #set up an engima for testing
                    rotors=config['rotors'],
                    reflector='B',
                    ring_settings=config['ring-settings'],
                    plugboard_settings=try_out_plugboard)
            decypher_enigma.set_display(config['display'])
            decyphered_text = decypher_enigma.process_text(cypher_text) #try to decypher the cypher_text
            if(decyphered_text == plain_text):
                print('-----------------------------------------------------------------------------------------------------------')
                print('FOUND SOLUTION! - decyphered text: ' + decyphered_text)
                config['plugboard'] = try_out_plugboard
                print(config)
                print('-----------------------------------------------------------------------------------------------------------')
                return True
    print('--invalid configuration : ' + decyphered_text + ' config_plugboard: ' + config['plugboard'])


                    
        

brute_force_remaining_plugboard({'display': 'ABC', 'plugboard': 'CX EI FT MR NZ', 'rotors': 'I II III', 'ring-settings': 'A A A'}, 'SSZCJVTHRXZVRLEAVYC', 'GUTENTAGMEINNAMEIST')
#heuristic_determination_of_remaining_plugboard({'plugboards' : list(['CX EI FT LS MR NZ UW QY']), 'rotors':'I II III', 'ring-settings':'A A A', 'display':'PSY'}, 'AAKFOWKHUABKEQXNMKLJXHUTIWNLXZE', 'GUTENTAGMEINNAMEISTMAXRUEDINGER')

In [16]:
alphabet = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

def find_identifier(a: str, b: str):
    """This method determines the identifier for an edge (necessary, since AB and BA represent the same edge)

    Args:
        a (str): the first node of an edge
        b (str): the second node of an edge

    Returns:
        str: the identifier for the edge (nodes sorted in alphabetical order)
    """
    if(alphabet.index(a) < alphabet.index(b)):
        token = a + b
    else:
        token = b + a
    return token

def hamming_distance(str1, str2):
    """
    Calculates the Hamming distance between two strings.
    
    :param str1: First string
    :param str2: Second string
    :return: Hamming distance
    """
    
    # Count the number of differing characters using a generator expression
    return sum(char1 != char2 for char1, char2 in zip(str1, str2))


def calc_hammingdistance_from_config(config: dict, crib:str, plain_text:str):
    decypher_enigma = EnigmaMachine.from_key_sheet( #set up an engima for testing
        rotors=config['rotors'],
        reflector='B',
        ring_settings=config['ring-settings'],
        plugboard_settings=config['plugboard'])
    decypher_enigma.set_display(config['display'])
    decyphered_text = decypher_enigma.process_text(crib)
    return hamming_distance(decyphered_text, plain_text)

# Example usage
str1 = "1011101"
str2 = "1001001"
print(f"Hamming distance: {hamming_distance(str1, str2)}")

Hamming distance: 2


In [17]:
def parse_steckerbrett(configuration):
    """
    Parses an Enigma plugboard configuration into a dictionary.
    
    :param configuration: A string representing the plugboard configuration (e.g., 'BO CI HT KY LP NU')
    :return: A dictionary mapping each letter to its paired letter
    """
    # Split the configuration into pairs
    pairs = configuration.split()
    
    # Create a dictionary for the mapping
    plugboard_mapping = {}
    
    for pair in pairs:
        # Map each letter to its counterpart
        plugboard_mapping[pair[0]] = pair[1]
        plugboard_mapping[pair[1]] = pair[0]
    
    return plugboard_mapping

# Example usage
config = 'BO CI HT KY LP NU'
plugboard = parse_steckerbrett(config)

print(f"Plugboard configuration: {plugboard}")

def create_plugboard_from_dict(mapping):
    """
    Converts a plugboard mapping dictionary back into a plugboard configuration string.
    
    :param mapping: A dictionary where each letter maps to its paired letter
    :return: A string representing the plugboard configuration (e.g., 'BO CI HT KY LP NU')
    """
    seen = set()  # To keep track of already processed letters
    pairs = []    # To store the pairs for the configuration string

    for key, value in mapping.items():
        # Ensure each pair is processed only once
        if key not in seen and value not in seen:
            pairs.append(f"{key}{value}")
            seen.add(key)
            seen.add(value)
    
    # Join the pairs with spaces to form the plugboard string
    return " ".join(pairs)

# Example usage
plugboard_mapping = {
    'B': 'O', 'O': 'B', 'C': 'I', 'I': 'C', 
    'H': 'T', 'T': 'H', 'K': 'Y', 'Y': 'K', 
    'L': 'P', 'P': 'L', 'N': 'U', 'U': 'N'
}

steckerbrett_string = create_plugboard_from_dict(plugboard_mapping)

print(f"Steckerbrett configuration: {steckerbrett_string}")

Plugboard configuration: {'B': 'O', 'O': 'B', 'C': 'I', 'I': 'C', 'H': 'T', 'T': 'H', 'K': 'Y', 'Y': 'K', 'L': 'P', 'P': 'L', 'N': 'U', 'U': 'N'}
Steckerbrett configuration: BO CI HT KY LP NU


In [18]:
from collections import deque
def reconstruct_plugboard(index_dict: dict, graph, rotor_string: str, start_display: str, start_letter:str, ring_settings: str, start_plugboard: str, enigma_settings_array, plain_text: str, cypher_text:str):
    cypher_text_list = list(cypher_text)
    current_plugboard = start_plugboard
    decypher_enigma = EnigmaMachine.from_key_sheet( #set up an engima for testing
        rotors=rotor_string,
        reflector='B',
        ring_settings=ring_settings,
        plugboard_settings=current_plugboard)
    plain_text_list = list(plain_text)
    
    visited = set()  # To track visited nodes
    queue = deque([start_letter])  # Queue for BFS (starting with the start node)
    plugged_letters = parse_steckerbrett(start_plugboard)
    unplugged_letters = set()
    

    while queue:
        current = queue.popleft()  # Dequeue the next node
        if current not in visited:
            visited.add(current)  # Mark the node as visited
            # Add unvisited neighbors to the queue
            decypher_enigma.set_display(start_display)
            for neighbor in graph.get(current, []): #iterate through all neighbors = all incident edges
                #set enigma display to correct position for edge (if there are multiple edges between two nodes, choose the first one)
                decypher_enigma.set_display(enigma_settings_array[index_dict[find_identifier(current, neighbor)][0]]) 
                resulting_char = decypher_enigma.process_text(current)
                plain_text_char = plain_text_list[index_dict[find_identifier(current, neighbor)][0]]
                cypher_text_char = cypher_text_list[index_dict[find_identifier(current, neighbor)][0]]
                
                expected_char = plain_text_char if current != plain_text_char else cypher_text_char
                
                if(resulting_char == expected_char and not resulting_char in plugged_letters):
                    unplugged_letters.add(resulting_char) #character must be unplugged
                elif(resulting_char != expected_char and not resulting_char in plugged_letters):
                    plugged_letters[resulting_char] = expected_char #add plug...
                    plugged_letters[expected_char] = resulting_char #...in both directions
                    if(len(plugged_letters) > 20):
                        return False, ""
                    current_plugboard = create_plugboard_from_dict(plugged_letters)
                    decypher_enigma = EnigmaMachine.from_key_sheet( #set up an engima for testing
                        rotors=rotor_string,
                        reflector='B',
                        ring_settings=ring_settings,
                        plugboard_settings=current_plugboard)
                elif(resulting_char != expected_char):
                    return False, ""
                if neighbor not in visited:
                    queue.extend(neighbor)
    return True, current_plugboard


#TEST-CODE:

import string
test_cypher_text = 'ITXBSSLXTGTUYAHIYN'
plain_text = 'LOREMIPSUMDOLORSIT'
start_display = 'KYI'
graph = {letter: [] for letter in string.ascii_uppercase}
cypher_text_list = list(test_cypher_text)
plain_text_list = list(plain_text)
index_dict = {}
for i in range(0, len(test_cypher_text)):
    graph[cypher_text_list[i]].append(plain_text_list[i])
    graph[plain_text_list[i]].append(cypher_text_list[i])
    #index_list.append(cypher_text_list[i] + plain_text_list[i])
    identifier = find_identifier(cypher_text_list[i], plain_text_list[i])
    if(not identifier in index_dict):
        index_dict[identifier] = list([i])
    else:
        index_dict[identifier].append(i)
print("graph: " + str(graph))
reconstruct_plugboard(index_dict, graph, 'I II III', start_display, 'L', 'A A A', 'LS', find_enigma_display_positions_to_index('I II III', plain_text, start_display), plain_text, test_cypher_text)

graph: {'A': ['O'], 'B': ['E'], 'C': [], 'D': ['T'], 'E': ['B'], 'F': [], 'G': ['M'], 'H': ['R'], 'I': ['L', 'S', 'S', 'Y'], 'J': [], 'K': [], 'L': ['I', 'P', 'Y'], 'M': ['S', 'G'], 'N': ['T'], 'O': ['T', 'U', 'A'], 'P': ['L'], 'Q': [], 'R': ['X', 'H'], 'S': ['M', 'I', 'X', 'I'], 'T': ['O', 'U', 'D', 'N'], 'U': ['T', 'O'], 'V': [], 'W': [], 'X': ['R', 'S'], 'Y': ['L', 'I'], 'Z': []}


(True, 'LS EI OP QY RM CX')

In [19]:
import string
import copy

import itertools
all_rotors = ['I', 'II', 'III', 'IV', 'V']


alphabet = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')


def get_plug(a: str, b: str, c:str, reconstruction:str):
            #this method reconstructs a plugboard connection from the letters seen during the loop iteration
            reconstruction = list(reconstruction)[-1]
            if(b==reconstruction):
                return a+c
            elif(c==reconstruction):
                return a+b
            else:
                return "[error: " + a + b + c + reconstruction + "]"

def run_turing_bombe(cycles: list, plain_text: str, cypher_text: str, graph:list, index_dict: dict, brute_force_right_ring_setting=False):
        print('SEARCHING FOR VALID ENIGMA CONFIGURATIONS')
        rotor_combinations = itertools.permutations(all_rotors, 3)
        number_trials = [0] * len(cycles)
        possible_configurations = list()
        for rotor_combination in rotor_combinations: #TODO: Richtigstellen
            rotor_selection = " ".join(rotor_combination) #TODO: Richtigstellen
            for last_ring in list(alphabet) if brute_force_right_ring_setting else ['A']:
                    brute_force_right_ring_setting=False
                    ring_settings = 'A A ' + last_ring
                    print('Running: Rotors: ' + rotor_selection + ' Ring Settings: ' + ring_settings)
                    display_iterator_enigma = EnigmaMachine.from_key_sheet(
                        rotors=rotor_selection,
                        reflector='B',
                        ring_settings=ring_settings,
                        plugboard_settings='')
                    display_iterator_enigma.set_display('AAA')
                    initial = True
                    while(initial or display_iterator_enigma.get_display() != 'AAA'):
                        enigma_display_array = find_enigma_display_positions_to_index(rotor_selection, plain_text, display_iterator_enigma.get_display())
                        valid_plugboard_for_display_exists = False
                        valid_plugboards_list = list()
                        for j in range(0, len(cycles)):
                            number_trials[j] += 1
                            valid_plugboards_list.append(list())

                            #List of the enigma configurations for every cycle edge
                            enigma_display_positions = [enigma_display_array[cycles[j]["index_list"][i]] for i in range(0, len(cycles[j]["index_list"]))]
                            
                            #determine the letter with which we start in the loop
                            start_letter = plain_text[cycles[j]["index_list"][0]] if plain_text[cycles[j]["index_list"][0]] == cycles[j]["cycle"][0] else cypher_text[cycles[j]["index_list"][0]]
                            for a in alphabet: 
                                plug_board = a + start_letter
                                if(plug_board==a + a):
                                    plug_board = ''
                                em = EnigmaMachine.from_key_sheet(
                                    rotors=rotor_selection,
                                    reflector='B',
                                    ring_settings=ring_settings,
                                    plugboard_settings=plug_board)
                                em.set_display(display_iterator_enigma.get_display())

                                l = list(start_letter)
                                for i in range(0, len(cycles[j]["index_list"])):
                                    em.set_display(enigma_display_positions[i])
                                    l.append(em.key_press(l[-1]))

                                if(l[0] == l[-1]):
                                    valid_plugboard, plugboard_reconstruction = reconstruct_plugboard(index_dict, graph, rotor_selection, display_iterator_enigma.get_display(), start_letter, ring_settings, plug_board, enigma_display_array, plain_text, cypher_text)
                                    if(valid_plugboard):
                                        valid_plugboards_list[-1].append(plugboard_reconstruction)
                                    #print(str(valid_plugboards_list) + " cycles: " + str(j) + "dispit " + display_iterator_enigma.get_display())

                            valid_plugboard_for_display_exists, valid_plugboards_list = check_plugboard_configurations(valid_plugboards_list)
                            if(not valid_plugboard_for_display_exists):
                                break
                                
                                
                        if valid_plugboard_for_display_exists:
                            for valid_pb in valid_plugboards_list[0]:
                                config = {'rotors' : rotor_selection,'ring-settings' : ring_settings, 'display' : display_iterator_enigma.get_display(), 'plugboard' : valid_pb}
                                config['hamming-distance'] = calc_hammingdistance_from_config(config, cypher_text, plain_text)
                                possible_configurations.append(config)
                                print(config)
                        display_iterator_enigma.key_press('A')
                        initial = False

        print("SEARCH FOR ENIGMA CONFIGURATIONS FINISHED")
        print("number of possible configurations: " + str(len(possible_configurations)))
        print('number of tried out configurations [per cycle]: ' + str(number_trials))
        print("------------------------------------------------------------------")
        return possible_configurations


def break_enigma(cypher_text: str, plain_text: str, brute_force_right_ring_setting=False):
    if(len(cypher_text) != len(plain_text)):
            print('Error: Different length of cypher_text and plain text!')
            return
    print("CRIB:       " + plain_text)
    print("CYPHERTEXT: " + cypher_text)
    print("TURNOVERPROBABILITY: " + str(round(min(len(cypher_text) / 26, 1) * 100, 2)) + '%')
    print("TURING-BOMBE IS RUNNING...")
    cypher_text = cypher_text[:min(35, len(cypher_text))] #limit length of cypher_text and plain text
    plain_text = plain_text[:min(35, len(plain_text))]
    graph = {letter: [] for letter in string.ascii_uppercase}
    cypher_text_list = list(cypher_text)
    plain_text_list = list(plain_text)
    index_dict = {}
    for i in range(0, len(cypher_text)):
        graph[cypher_text_list[i]].append(plain_text_list[i])
        graph[plain_text_list[i]].append(cypher_text_list[i])
        identifier = find_identifier(cypher_text_list[i], plain_text_list[i])
        if(not identifier in index_dict):
            index_dict[identifier] = list([i])
        else:
            index_dict[identifier].append(i)
    print("graph: " + str(graph))
    
    bare_cycles = find_all_cycles(graph, k=5)

    if(len(bare_cycles) < 1):
        print('Sorry, no loops in crib and cypher text found!')
        return

    print("cycles: " + str(bare_cycles))
    print("edge indices: " + str(index_dict))
    #store cycles in a dictionary
    cycles = [{"cycle": cycle} for cycle in bare_cycles]
    for i in range(0, len(cycles)):
        cycles[i]["index_list"] = list()
        index_dict_pop = copy.deepcopy(index_dict)
        for node in range(1, len(cycles[i]["cycle"])):
            #create a list, which contains for every edge in the cycle the corresponding index where the two letters are translated into each other
            cycles[i]["index_list"].append(index_dict_pop[find_identifier(cycles[i]["cycle"][node - 1], cycles[i]["cycle"][node])].pop(0))
        #add the last edge
        cycles[i]["index_list"].append(index_dict_pop[find_identifier(cycles[i]["cycle"][-1], cycles[i]["cycle"][0])].pop(0))

    #get find possible enigma configurations and sort them by the hamming distance between crib and cyphertext
    possible_configurations = sorted(run_turing_bombe(cycles, plain_text_list, cypher_text_list, graph, index_dict), key=lambda d: d['hamming-distance'])

    print("CHECKING ALL IDENTIFIED CONFIGURATIONS")
    for config in possible_configurations: #try out all configurations, starting with the lowest hamming-distance
        success = brute_force_remaining_plugboard(config, cypher_text, plain_text)
        if success:
            return


In [20]:
plain_text = "WEATHERISCLEAR"
m1 = EnigmaMachine.from_key_sheet(
       rotors='I II III',
       reflector='B',
       ring_settings="A A A",
       plugboard_settings='OP TF SL RM QY IE DJ ZN CX UW')

m1.set_display('AAA')
code1 = m1.process_text(plain_text)

break_enigma(cypher_text=code1, plain_text=plain_text)

CRIB:       WEATHERISCLEAR
CYPHERTEXT: NPNFRSPQAMDCBM
TURNOVERPROBABILITY: 53.85%
TURING-BOMBE IS RUNNING...
graph: {'A': ['N', 'S', 'B'], 'B': ['A'], 'C': ['M', 'E'], 'D': ['L'], 'E': ['P', 'S', 'C'], 'F': ['T'], 'G': [], 'H': ['R'], 'I': ['Q'], 'J': [], 'K': [], 'L': ['D'], 'M': ['C', 'R'], 'N': ['W', 'A'], 'O': [], 'P': ['E', 'R'], 'Q': ['I'], 'R': ['H', 'P', 'M'], 'S': ['E', 'A'], 'T': ['F'], 'U': [], 'V': [], 'W': ['N'], 'X': [], 'Y': [], 'Z': []}
cycles: [['C', 'M', 'R', 'P', 'E']]
edge indices: {'NW': [0], 'EP': [1], 'AN': [2], 'FT': [3], 'HR': [4], 'ES': [5], 'PR': [6], 'IQ': [7], 'AS': [8], 'CM': [9], 'DL': [10], 'CE': [11], 'AB': [12], 'MR': [13]}
SEARCHING FOR VALID ENIGMA CONFIGURATIONS
Running: Rotors: I II III Ring Settings: A A A
{'rotors': 'I II III', 'ring-settings': 'A A A', 'display': 'AAA', 'plugboard': 'XC RM IE OP LS ZN UW', 'hamming-distance': 2}
{'rotors': 'I II III', 'ring-settings': 'A A A', 'display': 'BNY', 'plugboard': 'HK BY JE IR LP XS OA DN', 'hamming-di

In [21]:
break_enigma("ODHKBI", "MONDAY")

CRIB:       MONDAY
CYPHERTEXT: ODHKBI
TURNOVERPROBABILITY: 23.08%
TURING-BOMBE IS RUNNING...
graph: {'A': ['B'], 'B': ['A'], 'C': [], 'D': ['O', 'K'], 'E': [], 'F': [], 'G': [], 'H': ['N'], 'I': ['Y'], 'J': [], 'K': ['D'], 'L': [], 'M': ['O'], 'N': ['H'], 'O': ['M', 'D'], 'P': [], 'Q': [], 'R': [], 'S': [], 'T': [], 'U': [], 'V': [], 'W': [], 'X': [], 'Y': ['I'], 'Z': []}
Sorry, no loops in crib and cypher text found!


In [22]:
aaa = EnigmaMachine.from_key_sheet(
    rotors='I II III',
    reflector='B',
    ring_settings="A A A",
    #plugboard_settings='BN DR ES FT GZ HU IJ KO ML')
    plugboard_settings='EI FT MR UW')

aaa.set_display('HSP')
aaa.process_text('FQYNFJZQXMJTMIOWKABPLGM')


'WWHKEWOMPISHTFLERMPMKAG'

In [23]:
from enigma.machine import EnigmaMachine

# setup machine according to specs from a daily key sheet:

machine = EnigmaMachine.from_key_sheet(
       rotors='I II III',
       reflector='B',
       ring_settings="A A A",
       plugboard_settings='QA WS')

# set machine initial starting position
machine.set_display('AAA')

# decrypt the message key
print(machine.process_text('AAA'))

CNV


__Tests für Ringstellungen__

In [24]:
m1 = EnigmaMachine.from_key_sheet(
       rotors='III II I',
       reflector='B',
       ring_settings="A A C",
       plugboard_settings='AW ES RD TF ZG UH IJ OK BN ML')

m2 = EnigmaMachine.from_key_sheet(
       rotors='III II I',
       reflector='B',
       ring_settings="A A A",
       plugboard_settings='AW ES RD TF ZG UH IJ OK BN ML')


m1.set_display('AAY')
print(m1.process_text('AAAAA'))
m2.set_display('AAW')
print(m2.process_text('AAAAA'))

HDXLN
HDXLN
